In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), "Chatbot-Agent/src")))

In [ ]:
import os
import getpass
from dataclasses import dataclass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


def setup_env():
    _set_env("TAVILY_API_KEY")
    os.environ["TOKENIZERS_PARALLELISM"] = "true"

    _set_env("LANGSMITH_API_KEY")
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "local-llm-rag-llama-3.2"

setup_env()

from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:3b-instruct-fp16", temperature=0)


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever the node is used.
graph_builder.add_node("Chatbot", chatbot)
graph_builder.add_edge(START, "Chatbot")
graph_builder.add_edge("Chatbot", END)
graph = graph_builder.compile()


ValueError: Node `Chatbot` already present.

In [ ]:
graph_builder.add_edge(START, "Chatbot")
graph_builder.add_edge("Chatbot", END)
graph = graph_builder.compile()
